In [2]:
import gym
import numpy as np
import os
import gym
import pickle
import argparse
import numpy as np
from collections import deque

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter 
#from utils.utils import *
#from utils.zfilter import *
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Actor(nn.Module):
    """
    
    RNN or LSTM, sequence to sequence
    
    input : state 6x 100 matrix, where 6 tokens, 100 dimensional embeddings
    
    
    actor outputs:
    
    for each sequence in the output, it is going produce a mu of size 1x100 (embedding shape), and then a
    corresponding standard deviation:
    
    
    Ex: embedding = 3:
    
    
    ["I","love","Rahul"] = [[.2,.31,.4],[.23,.1,.41],[-.75,-.52,.51]] = state
    
    
    mu, std = actor(state)
    
    
    GET ACTION
    mu = [[.21,.29,.45],[.221,.13,.421],[-.7,-.22,.58]]
    whatever we designate as action sequence length
    
    std = [[.01,.01,.01],[.01,.01,.01],[.01,.01,.01]]
    action = [[gauss(mean=.21,std=.01),gauss(.29,.01),gauss(.45,.01] ...]]
    
    TODO: lstm, attention 
    
    basic 
    
    seq -> encoder (lstm, attention, layer norm) -> decoder (lstm, attention, layer norm) -> seq
    
    """
    def __init__(self, num_inputs, num_outputs=100, hidden_size): # 100 dim output, continuous draw here. 
        super(Actor, self).__init__()
     #   self.RNN1 = nn.RNN(num_inputs, hidden_size)
      #  self.fc2 = nn.Linear(hidden_size, hidden_size)
      #  self.fc3 = nn.Linear(hidden_size, num_outputs)
        
     #   self.fc3.weight.data.mul_(0.1)
     #   self.fc3.bias.data.mul_(0.0)

    def forward(self, x):
       # x = torch.tanh(self.fc1(x))
      #  x = torch.tanh(self.fc2(x))
     #   mu = self.fc3(x)
     #   logstd = torch.zeros_like(mu)
        std = torch.exp(logstd)
        return mu, std


class Critic(nn.Module):
    """
    
    seq -> encoder (lstm, attention, layer norm) -> latent -> MLP so that it becomes one value, which is used to estimate
    Actor critic shiz
    """
    def __init__(self, num_inputs, hidden_size):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(num_inputs, hidden_size)

        self.fc3.bias.data.mul_(0.0)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        v = self.fc3(x)
        return v
    
    
#class ActorCritic()

    # within forward loop, after encoder, 


class Discriminator(nn.Module):
    """
    
    (state , action) -> discrimator -> prob
    
    
    what is prob? probability that this action taken with a particular state was done by an expert aka our real data, 
    or is what the output of action = actor(state) gave. 
    
    seq2 value 
    
    '
    '
    """
    def __init__(self, num_inputs, hidden_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.LSTM1(num_inputs, hidden_size)
        do

      #  self.fc3.weight.data.mul_(0.1)
     #   self.fc3.bias.data.mul_(0.0)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        prob = torch.sigmoid(self.fc3(x))
        return prob

SyntaxError: non-default argument follows default argument (<ipython-input-3-0aa871040d81>, line 2)

In [4]:
#code for training 
import torch
import numpy as np
from utils.utils import get_entropy, log_prob_density

def train_discrim(discrim, memory, discrim_optim, demonstrations, discrim_update_num, batch_size, clip_param):
    """
    Training the discriminator. 

    Use binary cross entropy to classify whether or not a sequence was predicted by the expert (real data) or actor. 
    """
    memory = np.array(memory)  # s a r s' tuple
    states = np.vstack(memory[:, 0]) 
    actions = list(memory[:, 1]) #actions taken by actor/policy

    states = torch.Tensor(states)
    actions = torch.Tensor(actions)
        
    criterion = torch.nn.BCELoss() # classify
 
    for _ in range(discrim_update_num):
        
        learner = discrim(torch.cat([states, actions], dim=1)) #pass (s,a) through discriminator
        demonstrations = torch.Tensor([states, expert_actions]) # pass (s,a) of expert through discriminator
        index = torch.randperm(demonstrations.shape[0])
        demonstrations_batch = demonstrations[index,:][:int(batch_size)] # batch of expert samples. Initially this was for the entire dataset, which might be too big. 
        expert = discrim(demonstrations_batch) #discrimator "guesses" whether or not these 
        # actions came from expert or learner
        discrim_loss = criterion(learner, torch.ones((states.shape[0], 1))) + \
                        criterion(expert, torch.zeros((demonstrations_batch.shape[0], 1)))
                # discrim loss: predict agent is all wrong, get as close to 0, and predict expert is 1, getting as close to 1 as possible. 
        discrim_optim.zero_grad() # gan loss, it tries to always get it right. 
        discrim_loss.backward()
        discrim_optim.step()

        # take these steps, do it however many times specified. 

    expert_acc = ((discrim(demonstrations_batch) < 0.5).float()).mean() #how often it realized the fake examples were fake
    learner_acc = ((discrim(torch.cat([states, actions], dim=1)) > 0.5).float()).mean() #how often if predicted expert correctly. 

    return expert_acc, learner_acc # accuracy, it's the same kind, but because imbalanced better to look at separately. 
 

def train_actor_critic(actor, critic, memory, actor_optim, critic_optim, actor_critic_update_num, batch_size, clip_param):
    """
    Take a PPO step or two to improve the actor critic model,  using GAE to estimate returns. 
    
    In our case each trajectory it most one step, so the value function will have to do. 
    
    
    """
    memory = np.array(memory) 
    # tuple of a regular old RL problem, but now reward is what the discriminator says. 
    states = np.vstack(memory[:, 0]) 
    actions = list(memory[:, 1]) 
    rewards = list(memory[:, 2])  #IRL Rewards
    masks = list(memory[:, 3]) 

    # compute value of what happened, see if what we can get us better. 
    old_values = critic(torch.Tensor(states))
    #GAE aka estimate of Value + actual return roughtly 
    returns, advants = get_gae(rewards, masks, old_values, gamma, lamda)
    
    # pass states through actor, get corresponding actions
    mu, std = actor(torch.Tensor(states))
    # new mus and stds? 
    old_policy = log_prob_density(torch.Tensor(actions), mu, std) # sum of log probability
    # of old actions

    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)

    for _ in range(actor_critic_update_num):
        np.random.shuffle(arr)

        for i in range(n // batch_size): 
            batch_index = arr[batch_size * i : batch_size * (i + 1)]
            #batch_index = torch.LongTensor(batch_index)
            
            inputs = torch.Tensor(states)[batch_index]
            actions_samples = torch.Tensor(actions)[batch_index]
            returns_samples = returns.unsqueeze(1)[batch_index]
            advants_samples = advants.unsqueeze(1)[batch_index]
            oldvalue_samples = old_values[batch_index].detach()
        
        
            values = critic(inputs) #
            clipped_values = oldvalue_samples + \
                             torch.clamp(values - oldvalue_samples,
                                         -clip_param, 
                                         clip_param)
            critic_loss1 = criterion(clipped_values, returns_samples)
            critic_loss2 = criterion(values, returns_samples)
            critic_loss = torch.max(critic_loss1, critic_loss2).mean()

            loss, ratio, entropy = surrogate_loss(actor, advants_samples, inputs,
                                         old_policy.detach(), actions_samples,
                                         batch_index)
            clipped_ratio = torch.clamp(ratio,
                                        1.0 - clip_param,
                                        1.0 + clip_param)
            clipped_loss = clipped_ratio * advants_samples
            actor_loss = -torch.min(loss, clipped_loss).mean()
            #print(actor_loss,critic_loss,entropy)
           # return actor_loss, critic_loss, entropy
            loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy #entropy bonus to promote exploration.

            actor_optim.zero_grad()
            loss.backward()
            actor_optim.step()

           # critic_optim.zero_grad()
           # loss.backward() 
            critic_optim.step()

def get_gae(rewards, masks, values, gamma, lamda):
    """
    How much better a particular action is in a particular state. 
    
    Uses reward of current action + value function of that state-action pair, discount factor gamma, and then lamda to compute. 
    """
    rewards = torch.Tensor(rewards)
    masks = torch.Tensor(masks)
    returns = torch.zeros_like(rewards)
    advants = torch.zeros_like(rewards)
    
    running_returns = 0
    previous_value = 0
    running_advants = 0

    for t in reversed(range(0, len(rewards))): #for LL, only ever one step :-)
        running_returns = rewards[t] + (gamma * running_returns * masks[t])
        returns[t] = running_returns

        running_delta = rewards[t] + (gamma * previous_value * masks[t]) - \
                                        values.data[t]
        previous_value = values.data[t]
        
        running_advants = running_delta + (gamma * lamda * \
                                            running_advants * masks[t])
        advants[t] = running_advants

    advants = (advants - advants.mean()) / advants.std()
    return returns, advants

def surrogate_loss(actor, advants, states, old_policy, actions, batch_index):
    """
    The loss for PPO. Re-run through network, recomput policy from states
    and see if this surrogate ratio is better. If it is, use as proximal policy update. It's very close to prior policy, but def better. 
    
    Not sure this actually works though. Should not the new mu and stds be used to draw,
    
        When do we use get_action? Only once in main, I think it should be for all? 
    """
    mu, std = actor(states)
    new_policy = log_prob_density(actions, mu, std)
    old_policy = old_policy[batch_index]

    ratio = torch.exp(new_policy - old_policy)
    surrogate_loss = ratio * advants
    entropy = get_entropy(mu, std)

    return surrogate_loss, ratio, entropy

ModuleNotFoundError: No module named 'utils'

Below are the hyperparameters. It's going to be a bitch, but we need to find a way to optimize these. 

Some heuristics might help. 

In [ ]:
# Normally args but not here :-)
env_name = # Dialog
load_model = None
train_discrim_flag = True # starts here. 


seed = 0 # should try 3
hidden_size = 128 # should try a few, also add more layers? 
learning_rate = 3e-4 # try a few
clip_param = .2 # try a few
discrim_update_num = 2 # try a few
actor_critic_update_num = 10 # try a few
l2_rate = 1e-3 # weight decay # try a few
total_sample_size = 256 # total num of state-actions to collect before learning 
batch_size = 32

# That's about 9 values. 9^3 is 729, and a shit load more than we're able to try




render = False #X X
gamma = 0.99 # no longer used ! 
lamda = .98 # lambda is used for future steps in GAE. Also only one 


# Not sure, maybe go for iters
suspend_accu_exp = 1. # do not need to be this high typically, but seems likely it has to be for a simple env like mountain car cont.
suspend_accu_gen = 1.
max_iter_num = 500

In [ ]:
# one last setup before main
logdir = 'logs'

env = gym.make(env_name)
    
env.seed(seed)
torch.manual_seed(seed)

num_inputs = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]
running_state = ZFilter((num_inputs,), clip=5) # huh? 
# oh wow. ZFilter is exactly what I do in capstone project, removing "badtimes"

print('state size:', num_inputs) 
print('action size:', num_actions)

#load agent stuff 
actor = Actor(num_inputs, num_actions, hidden_size)
critic = Critic(num_inputs, hidden_size)
discrim = Discriminator(num_inputs + num_actions, hidden_size)

actor_optim = optim.Adam(actor.parameters(), lr=learning_rate)
critic_optim = optim.Adam(critic.parameters(), lr=learning_rate, 
                          weight_decay=l2_rate) 
discrim_optim = optim.Adam(discrim.parameters(), lr=learning_rate)

In [ ]:
# load demonstrations
#expert_demo, _ = pickle.load(open('./expert_demo/expert_demo.p', "rb"))
#demonstrations = np.load('/Users/noahkasmanoff/Desktop/Projects/lets-do-irl/mountaincar/app/expert_demo/expert_demo.npy')
print("demonstrations.shape", demonstrations.shape)

writer = SummaryWriter(logdir)

#if you aren't starting from scratch, load in this 
if load_model is not None:
    saved_ckpt_path = os.path.join(os.getcwd(), 'save_model', str(load_model))
    ckpt = torch.load(saved_ckpt_path)

    # initialize everything
    actor.load_state_dict(ckpt['actor'])
    critic.load_state_dict(ckpt['critic'])
    discrim.load_state_dict(ckpt['discrim'])

    running_state.rs.n = ckpt['z_filter_n']
    running_state.rs.mean = ckpt['z_filter_m']
    running_state.rs.sum_square = ckpt['z_filter_s']

    print("Loaded OK ex. Zfilter N {}".format(running_state.rs.n))

# if no old model no worries, start training. 
episodes = 0
for iter in range(max_iter_num):
    # for i total trajectories 
    actor.eval(), critic.eval()
    memory = deque()

    steps = 0
    scores = []

    while steps < total_sample_size: 
        # sample trajectories  (batch size)
        state, expert_action = env.reset() #Potential TODO: expert demonstrations saved somewhere else? 
        score = 0

        # TODO: Do we need this anymore? 
        state = running_state(state) #uh.. again ZFilter related, cleans the state  
        
        epsteps = 0
        done = False
        while not done: 
            epsteps += 1
            #run through environment
            if render: 
                env.render()

            steps += 1

            #TODO: forward pass through RNN
            mu, std = actor(torch.Tensor(state).unsqueeze(0)) #pass state through actor network
            action = get_action(mu, std)[0] #compute random action
             done = env.step(action) #take a step
            irl_reward = get_reward(discrim, state, action) #infer what the reward of this action is based on discriminator's get reward 

            if done:
                mask = 0
            else:
                mask = 1 #if done, save this, 

            memory.append([state, action, irl_reward, mask])
            
            #TODO: no longer exists :-)
            #next_state = running_state(next_state) #save cleaned next state
            #state = next_state #and set to current state, 

            score += irl_reward #add total reward
           # print("IRL Reward=",irl_reward)
            if done:
                break
            #actual sampling done here 


        
        episodes += 1
        scores.append(score)
        print("steps in ep:", epsteps)
    score_avg = np.mean(scores) #how this model did, 
    print('{}:: {} episode score is {:.2f}'.format(iter, episodes, score_avg))
    writer.add_scalar('log/score', float(score_avg), iter) #logg

    actor.train(), critic.train(), discrim.train() #now train 
    if train_discrim_flag: #if this batch optimizes discrim/reward, 
        # for training the discriminator, classify where state-action pair came from. 
        expert_acc, learner_acc = train_discrim(discrim, memory, discrim_optim, demonstrations, discrim_update_num, batch_size, clip_param) # see comments in train_model. 
        print("Expert: %.2f%% | Learner: %.2f%%" % (expert_acc * 100, learner_acc * 100))
        if expert_acc > suspend_accu_exp and learner_acc > suspend_accu_gen:
            print("Now it will only train the policy, seeing as it is good enough at finding the differences between learner and expert trajectories.")
            train_discrim_flag = False #now train only policy, vanilla RL
            render = True
    #for training actor critic 
    
    # PPO operation, 
    train_actor_critic(actor, critic, memory, actor_optim, critic_optim, actor_critic_update_num, batch_size, clip_param) # no output, see comments in train_model 

    if iter % 100:
        score_avg = int(score_avg)

        model_path = os.path.join(os.getcwd(),'save_model')
        if not os.path.isdir(model_path):
            os.makedirs(model_path)

        ckpt_path = os.path.join(model_path, 'ckpt_'+ str(score_avg)+'.pth.tar')

        save_checkpoint({
            'actor': actor.state_dict(),
            'critic': critic.state_dict(),
            'discrim': discrim.state_dict(),
            'z_filter_n':running_state.rs.n,
            'z_filter_m': running_state.rs.mean,
            'z_filter_s': running_state.rs.sum_square,
           # 'args': args,
            'score': score_avg
        }, filename=ckpt_path)